In [1]:
import numpy as np
import seaborn as sns
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

# https://biopython.org/docs/1.74/api/Bio.SVDSuperimposer.html
from Bio.SVDSuperimposer import SVDSuperimposer

from numpy import array, dot, set_printoptions

In [2]:
# link to openmm manual for how to set up a minimizer
# http://docs.openmm.org/7.2.0/userguide/application.html

In [62]:
def get_rot_and_trans(subtraj_A,subtraj_B):
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms


def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

In [4]:
# Load H-NS dimers
loc_dimers = './data/dimer_pdbs/'
dimers = md.join([md.load(loc_dimers+f'run{i}.pdb') for i in range(0,16)])

# Load H-NS DBD attached to DNA
loc_complexes = './data/complex_traj/'
complexes = md.load(loc_complexes+f'DBD_complex.xtc', top=loc_complexes+f'DBD_complex.pdb')

In [40]:
traj = dimers

In [54]:
selection = traj.top.select('resid 111 112 and (name CA or name N)')
view = nv.show_mdtraj(traj)

view.add_representation('ball+stick',selection=selection)
view

NGLWidget(max_frame=15)

In [28]:
# Select dimer frame and which monomer (0 or 1)
idx = 2
chain =  0

# Extract xyz coordinates of first frame complex and idx dimer
A = complexes[0].remove_solvent()[0]
B = dimers[idx].remove_solvent()[0]

view = nv.show_mdtraj(A.stack(B))
view

NGLWidget()

In [29]:
#select atom indices of DBD forming an complex with DNA
selection_A = A.top.select(f'(resid {41} to {41+59}) and (backbone)') 

#select atom indices of DBD from S1S1 of monomer 'chain' this synthax unfortuntaly only works for chain A/0
selection_B = B.top.select(f'(chainid {chain} and resid {137-58} to {137}) and (backbone)')

# check if selections contain equal number of atoms
if len(selection_A) == len(selection_B):
    print(len(selection_A),len(selection_B))
    pass
else:
    print('no good')
    print(len(selection_A),len(selection_B))
    
# create trajs containing only the selections
subtraj_A = A.atom_slice(selection_A)
subtraj_B = B.atom_slice(selection_B)

# obtain instructions to rotate and translate B on A based on substraj structures
rot, tran,rms = get_rot_and_trans(subtraj_A,subtraj_B)
print(rms)

# do the superimposition of B on A and subsitute old with new xyz of B
new_B = apply_superimposition(B, rot, tran)

# remove selection from A and add new B
selection_to_delete = A.top.select(f'chainid 2') 
C = A.atom_slice([at.index for at in A.top.atoms if at.index not in selection_to_delete]).stack(new_B)

232 232
1.0041893237879114


In [30]:
# show merged fit
view = nv.show_mdtraj(A.stack(new_B))
view

NGLWidget()

In [14]:
# show cleaned final
view = nv.show_mdtraj(C)
view

NGLWidget()

In [63]:
def fit_B_on_A(A, B, selection_A, selection_B, selection_to_delete=None):
    
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(selection_A)
    subtraj_B = B.atom_slice(selection_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)
    
    # do the superimposition of B on A and subsitute old with new xyz of B
    new_B = apply_superimposition(B, rot, tran)
    
    if selection_to_delete == 'DBD_complex':
        # remove selection from A and add new B
        selection_to_delete = A.top.select(f'chainid 2') 
        C = A.atom_slice([at.index for at in A.top.atoms if at.index not in selection_to_delete]).stack(new_B)
        
    else:
         C = A.stack(new_B)
            
    return C, rms

In [16]:
selection = 'QGR'

if selection == 'DBD':
    #select atom indices of DBD forming an complex with DNA
    selection_A = A.top.select(f'(resid {41} to {41+59}) and (backbone)') 

    #select atom indices of DBD from S1S1 of monomer 'chain'
    selection_B = B.top.select(f'(chainid {chain} and resid {137-58} to {137}) and (backbone)')
    
elif selection == 'QGR':
    n = 15
    #select atom indices of DBD forming an complex with DNA
    selection_A = A.top.select(f'resid {73-n} to {75+n}') 

    #select atom indices of DBD from S1S1 of monomer 'chain'
    selection_B = B.top.select(f'chainid {chain} and resid {111-n} to {113+n}')

In [17]:
view = nv.show_mdtraj(A)
view.add_representation('licorice',selection_A)
view

NGLWidget()

In [18]:
view = nv.show_mdtraj(B)
view.add_representation('licorice',selection_B)
view

NGLWidget()

In [19]:
# check if selections contain equal number of atoms
if len(selection_A) == len(selection_B):
    print(len(selection_A),len(selection_B))
    C, rms = fit_B_on_A(A,B,selection_A,selection_B,selection_to_delete=None)
    print(rms)
else:
    print('no good')
    print(len(selection_A),len(selection_B))
    
# show cleaned final
view = nv.show_mdtraj(C)
view

503 503
0.3999208104597928


NGLWidget()

## Grafting

In [276]:
# create small piece of protein
traj = dimers
top = traj.topology
selection = top.select('resid 60 to 79')
print(traj)
traj = dimers.atom_slice(selection)
view  = nv.show_mdtraj(traj.center_coordinates())

view

<mdtraj.Trajectory with 16 frames, 4390 atoms, 274 residues, without unitcells>


NGLWidget(max_frame=15)

In [277]:
traj.top.n_bonds

311

In [340]:
# split the small piece in 2 as a demo on how to "repair/graft" it to once piece again
traj_A = traj.atom_slice(traj.top.select('resid 0 to 11'))
traj_B = traj.atom_slice(traj.top.select('resid 10 to 20'))

# pick residues that overlap and that you will use for the fit
selection_A = traj_A.top.select('resid 10 11')
selection_B = traj_B.top.select('resid 0 1')

print(len(selection_A))
print(len(selection_B))

26
26


In [330]:
# # show piece A with atoms used in the fit
# view  = nv.show_mdtraj(traj_A)
# view.add_representation('licorice',traj_A.top.select('protein'))
# view.add_representation('licorice',selection_A,color='red')
# view

# # show piece B with atoms used in the fit
# view  = nv.show_mdtraj(traj_B)
# view.add_representation('licorice',traj_B.top.select('protein'))
# view.add_representation('licorice',selection_B,color='red')
# view

In [331]:
C = None

In [336]:
# Fit B on A with the overlap atoms, here I used frame zero from piece A and frame 2 from piece B, you can of course change this easily
C, rms = fit_B_on_A(traj_A[0], traj_B[5], selection_A, selection_B)
print(rms)
# still need to remove "overlap" because now it is twice in structure, orignal structure had only 20 residues, this one 22 as well as chainIDs
# you can actually do this before you merge the structures (easier than after merging [the stack command]) 
C.top

0.017694514283698816


<mdtraj.Topology with 2 chains, 22 residues, 336 atoms, 337 bonds at 0x232bd5210>

In [335]:
# show fitted complex
view = nv.show_mdtraj(C)
view

NGLWidget()

In [337]:
def get_rot_and_trans(subtraj_A,subtraj_B):
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
         even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms


def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def update_topology(C):
    top = C.top
    # Merge two tops (with two chains or more) to a top of one chain 
    out = md.Topology()
    c = out.add_chain()
    for chain in top.chains:

        for residue in chain.residues:
            r = out.add_residue(residue.name, c, residue.resSeq, residue.segment_id)
            for atom in residue.atoms:
                out.add_atom(atom.name, atom.element, r, serial=atom.serial)
    #     for bond in top.bonds:
    #         a1, a2 = bond
    #         out.add_bond(a1, a2, type=bond.type, order=bond.order)
    out.create_standard_bonds() #rare manier om bonds te maken, maar werkt
    C.top = out 
    return C

def fit_B_on_A(A, B, selection_A, selection_B, delete_overlap=False):
    
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(selection_A)
    subtraj_B = B.atom_slice(selection_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)
    
    # do the superimposition of B on A and subsitute old with new xyz of B
    new_B = apply_superimposition(B, rot, tran)

    
    if delete_overlap:
        # remove selection from A and add new B
        selection_to_delete = selection_A 
        new_A = A.atom_slice([at.index for at in A.top.atoms if at.index not in selection_to_delete])

        # merge superimposed structure and fit (A and B*) and update topology such that they share a chain
        C = update_topology(new_A.stack(new_B))
    else:
         C = A.stack(new_B)
            
    return C, rms



In [338]:
C = None
# Fit B on A with the overlap atoms, here I used frame zero from piece A and frame 2 from piece B, you can of course change this easily
C, rms = fit_B_on_A(traj_A[3], traj_B[10], selection_A, selection_B, delete_overlap=True)
print(rms)
C.top

0.010759860893914848


<mdtraj.Topology with 1 chains, 22 residues, 328 atoms, 323 bonds at 0x232da0a00>

In [339]:
view = nv.show_mdtraj(C)
view.add_representation('licorice',selection=selection_A)
view

NGLWidget()